# Candy Crush in Python - Part 3 - Matching

In this part of the tutorial we will write the code to search for matches of 3 or more tiles in a row or column of our grid.

## Code Recap

So far we have our game board, a way of displaying the grid, and a function to swap two cells beside eachother:

In [ ]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display

def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)
    
def swap_items(board, first, second):    
    # Calculate the 'distance'
    dist = np.sum(np.abs(np.subtract(second, first)))
    # Two cells are beside eachother if the distance is 1 or 0
    is_beside = dist <= 1
    # Only swap if they are beside
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 

And we can use our code as follows:

In [ ]:
swap_board = np.array([
    [0, 1, 2],
    [3, 4, 0],
    [1, 2, 3]    
])

print("Before Swap:")
show_board(swap_board)

# Swap (0,0) with (0, 1)
swap_items(swap_board, (0,0), (0,1))

# Display the game board
print("After Swap:")
show_board(swap_board)

## Finding Matches in a Row

The object of the game is to get three or more gems in a row or a column, so we need some code that is able to find these matches on our game board. 

We will break this down into a few parts:
* For a given tile, count the number of tiles beside (to the right of) that have the same value
* For a given tile, count the number of tiles below it of the same value
* Go through every tile and use the above two functions to find all groups of 3 or longer

### Counting Matching Tiles to the Right

To count the number of cells to the right of a cell, we will write a function called horiz_match_length.

The function will take our board and a point (i.e. the row and column index) of the tile we want to check. It will return the number of tiles to the right that match (including the current tile - i.e. the number of tiles that are the same to the right plus 1).

#### Getting the row and column from our tuple

We need to know the row and column of our point, to do this we use `(r, c) = point`:

In [ ]:
point = (0, 1)
(r, c) = point
print("Row: " + str(r) + ", Column: " + str(c))

#### Looping through the cells in the row

We want to check each cell in the row of the point we are checking, to do this we use a for loop, starting at the next column `c+1`, until we reach the end of the row. 

To get the end of the row we can use the shape property of the board. The shape tells us how many rows and columns are on the board e.g.:

In [ ]:
board = np.array([
    [0,1,2,3],
    [4,0,1,2],
    [3,4,0,1]
])
print(board.shape)

This returns a tuple, so we can get the number of rows and columns using `(row_count, col_count) = board.shape`

In [ ]:
(row_count, col_count) = board.shape
print("Board Dimensions: " + str(row_count) + "x" + str(col_count))

So now we can loop over every cell starting at our column `c+1`:

In [ ]:
for i in range(c+1, col_count):
    print("Row: " + str(r) + ", Column: " + str(i) + " = " + str(board[r, i]))

#### Creating the Function

So now we can create our function, we start by setting some variables:
* Get the value of the point we are checking, put it in a variable called `first`
* Put the row and column of the point into `r` and `c`
* Put the number of rows and columns of the board into `row_count` and `col_count` (Though we only actually need col_count for this function)
* Create a variable `same_tile_count` and set it to 1 (we always have a match of length 1)

Our function then does the following:
* Loop over every cell starting at the one to the right of our point (`r`, `c+1`)
* Check if the tile has the same value as our `first` cell
* If it does have the same value, add 1 to `same_tile_count`
* If it does not have the same value, we have reached the end of the match return the current `same_tile_count`

In [ ]:
def horiz_match_length(board, point):
    
    # The value of the point we are checking
    first = board[point]
    
    # Get the row and column of the point we are checking
    (r, c) = point
    
    # Get the number of rows and columns
    (row_count, col_count) = board.shape
    
    # The count of how many of the same tile we have found in a row (we always have at least 1)
    same_tile_count = 1
    
    # We check each tile from the tile immediately to the right (c+1) to the end of the row 
    for i in range(c+1, col_count):
        # Check if the tile we are looking at is the same as the first
        if(board[r, i] != first):
            # Its not so just return the count we hae so far
            return same_tile_count
        else:
            # Add one to the count
            same_tile_count = same_tile_count + 1
            
    return same_tile_count

#### Using the Function

Lets test our function with some values, first lets create a board:

In [ ]:
count_horiz_board = np.array([
    [0, 1, 2, 3],
    [3, 4, 4, 4],
    [1, 1, 0, 3],
    [1, 2, 0, 2]    
])

show_board(count_horiz_board)

We can see from this that we have 3 ruby's in a row starting at row 1 column 1 or (1, 1) (Remember the first row and column is 0, not 1). We also have a row of two white diamonds starting at (2, 0). If we checked every tile in the grid, we would expect it to look like the following:

| | C0 | C1 | C2 | C3 |
|-|
| R0 | 1 | 1 | 1 | 1 |
| R1 | 1 | 3 | 2 | 1 |
| R2 | 2 | 1 | 1 | 1 |
| R3 | 1 | 1 | 1 | 1 |

So lets check a couple of these with our function:

In [ ]:
horiz_match_length(count_horiz_board, (0, 0))

In [ ]:
horiz_match_length(count_horiz_board, (1, 1))

We can use a loop and another array to check all positions and return an array with the count for each point on our board. We can create an empty array with the same shape as our board to store the counts using np.zeros:

In [ ]:
counts = np.zeros(count_horiz_board.shape, dtype=int)
print(counts)

And we can use a loop to check every cell in our grid:

In [ ]:
(row_count, col_count) = count_horiz_board.shape

for r in range(0, row_count):
    for c in range(0, col_count):
        counts[r,c] = horiz_match_length(count_horiz_board, (r, c))
        
print(counts)

## Finding Matches in a Column

Counting the number of tiles below is very much like counting the number of tiles to the left. We create a function that takes our game board, the row and column of the tile we want to check, and we loop through all the cells below in the same column until we find a different one. We then return the number of matching tiles we found:

In [ ]:
def vert_match_length(board, point):
    # Create our variables
    first = board[point]
    (r,c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1    
    
    # This time we check from the row immediately below to the last row
    # game_grid.shape[0] gives us the number of rows
    for i in range(r+1, row_count):
        if(board[i,c] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count   

Lets now test it with a game grid, and verify we get the results we expect.

In [ ]:
count_vert_board = np.array([
    [0, 1, 2, 3],
    [2, 4, 0, 4],
    [1, 3, 0, 3],
    [0, 2, 0, 3]    
])

show_board(count_vert_board)

We expect the following results:

| | C0 | C1 | C2 | C3 |
|-|
| R0 | 1 | 1 | 1 | 1 |
| R1 | 1 | 1 | 3 | 1 |
| R2 | 1 | 1 | 2 | 2 |
| R3 | 1 | 1 | 1 | 1 |

In [ ]:
(row_count, col_count) = count_vert_board.shape
vert_counts = np.zeros((row_count, col_count), dtype=int)

for r in range(0, row_count):
    for c in range(0, col_count):
        vert_counts[r,c] = vert_match_length(count_vert_board, (r, c))
        
print(vert_counts)

## Finding all Row and Column Matches

We are now ready to create a function that finds all matches on the board. This function will take a board, and a `required_matches`, which is the minimum number of the same type in a row we are looking for.

The function goes through each cell, and checks both the horizontal and vertical matches at that point. If it finds one that is long enough, it adds it to a list of matches.

At the end it returns a tuple with two lists, one for the horizontal matches and one for the vertical matches.

### Adding items to a list

For this function, since we will return two lists with the horizontal and vertical matches, we need to be able to add items to our list. We can use the `list.append(...)` function for this:

In [ ]:
l = [] # An empty list
print(l)

l.append(1)
l.append(2)

print(l)

Each 'match' we add to the list will be a tuple, with the row, column and the length of the match, e.g. the tuple `(2,1,3)` represents a match at point `(2,1)` (the 3rd row, 2nd column) of length `3`.

### Creating the function

Our function will take the following parameters:
* `board` - the board we want to find matches on
* `required_matches` - the minimum number of matched tiles to search for

We will first create the following variables:
* `horiz_matches` - a list to store the matches in the horizontal direction
* `vert_matches` - a list to store the matches in the vertical direction
* `row_count` and `col_count` to store the number of rows and columns in the board

And we will loop through each cell on the board and do the following:
* Create a point for the row and column called `point`
* Check the length of the match in the horizontal direction at that point, using our `horiz_match_length` function
* If the lenth of the match is greater or equal to our `required_matches` we add a match to our `horiz_matches` list
* Check the length of the match in the vertical direction at that point, using our `vert_match_length` function
* If the length of the match is greater or equal to our `required_matches` we add a match to our `vert_matches` list

After we have looped over every cell, we return a tuple with our horizontal and vertical matches:

In [ ]:
def find_matches(board, required_matches):
    # Create a list of horizontal matches
    horiz_matches = []
    # Create a list of vertical matches
    vert_matches = []
    
    (row_count, col_count) = board.shape
    
    # Loop through every row
    for r in range(row_count):
        # Loop through every column
        for c in range(col_count):
            point = (r,c)
            # Find the number of matched cells at this point
            horiz_len = horiz_match_length(board, point)
            
            # If the number of matched cells is greater than what we require, 
            #  then add to the list of horizontal matches
            if horiz_len >= required_matches:
                horiz_matches.append((r, c, horiz_len))  
            
            # Find the number of matching tiles below at this point
            vert_len = vert_match_length(board, point)
            
            # If the number of matched cells is greater than our required_count
            #  the add to the list of vertical matches
            if vert_len >= required_matches:
                vert_matches.append((r, c, vert_len))
                
    return (horiz_matches, vert_matches)

### Testing the Function

We can now test our function with a board that contains both a horizontal and vertical match:

In [ ]:
find_all_board = np.array([
    [2, 1, 2, 3],
    [2, 0, 0, 0],
    [2, 3, 0, 3],
    [4, 2, 0, 3]    
])

show_board(find_all_board)

In [ ]:
(h_matches, v_matches) = find_matches(find_all_board, 3)
print("Horizontal: "+ str(h_matches))
print("Vertical: " + str(v_matches))

This is telling us we have one horizontal match starting at (1,1), and two vertical matches, one starting at (0,0) and the other at (1,2). All three matches are of length 3.
We can also check for matches of 2 or more:

In [ ]:
(h_matches, v_matches) = find_matches(find_all_board, 2)
print("Horizontal: "+ str(h_matches))
print("Vertical: " + str(v_matches))

or for matches of 4 or more:

In [ ]:
(h_matches, v_matches) = find_matches(find_all_board, 4)
print("Horizontal: "+ str(h_matches))
print("Vertical: " + str(v_matches))

## Summary

In this lesson we learned how to find all matches of a particular length on our board. We created a function `find_matches` that takes a board and the number of matches we want and returns a tuple with a list of all the horizontal matches and a list of all the vertical matches. 

In the next section we will look at: [Removing Matches and Filling in Blank Spaces](Part4-RemoveAndReplace.ipynb)